# Installation H2 

Le système utilisé pendant les TME est H2.

In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger le pilote de H2

In [2]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-04-09 13:03:24--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,01s   

2024-04-09 13:03:24 (102 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-04-09 13:03:24--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,02s   

2024-04-09 13:03:2

In [3]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

## Démarrer le serveur SQL H2

In [5]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [6]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

## Fonctions utiles

In [7]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

In [9]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
            for attr in names:
                print(str(attr[:16]).ljust(16),end='|')
            print()
            for attr in names:
                print(''.ljust(17,'-'),end='')
            print()
            for ligne in cursor:
                for attr in ligne:
                    print(str(attr)[:16].ljust(16),end='|')
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

# TD et TME 6 : REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY
Ce TD/TME utilise les données contenues dans les fichiers **bd_jo_v1_H2.sql** et **bd_jo_v2_H2.sql**


## TD4 

Ce TD utilise les données contenues dans les fichiers **bd_jo_v1_H2.sql** 

BASE DE DONNÉES **JEUX OLYMPIQUES D'HIVER 2014** <br/>
On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des Jeux Olympiques d'Hiver Sotchi 2014 :<br/>


**ATHLETE** ( <u>NOM,PRENOM</u>,DATENAISSANCE,PAYS)<br/>
**RANGEPREUVE**( <u>NOMATH*,PRENOMATH*,EPREUVE,SPORT,CATEGORIE</u>,DATEDEBUT,DATEFIN,RANG)<br/>

La relation ATHLETE contient des informations concernant les athlètes ayant participé aux Jeux
Olympiques. Plus précisément : leur nom, leur prénom, leur date de naissance si elle est renseignée
(ou NULL sinon) ainsi que le pays qu'ils représentent.
La relation RANGEPREUVE renvoie pour chaque athlète identifie par NOMATH ET PRENOMATH et
chaque épreuve identifiée par SPORT, EPREUVE ET CATEGORIE (HOMMES OU FEMMES) à laquelle il a
participé, le RANG qu'il a obtenu. On pour chaque épreuve les dates de début et de fin (DATEDEBUT,
DATEFIN).
Lorsqu'un athlète participe à une épreuve puis est disqualifié, son rang n'est pas renseigné (NULL)



### Créer connexion et charger les données JO-V1

## Connexion à H2

In [11]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection closed
Connection to H2 DB successful


**!Vérifiez** que vous avez bien un fichier **jo_numero_port.mv.db** dans votre répertoire data

## Charger les données bd_jo_v1_H2.sql

In [14]:
schemafile=open("bd-jo-v1_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


### Requêtes

**ATTENTION :** 

**il ne faut PAS terminer les requêtes avec ';'**

**Utiliser simples quotes (') pour les chaînes de caractères, ne pas utiliser doubles quote (")** 

**Q1**. Les athlètes français.<br/>
Résultat : (92 lignes)

In [15]:
query="""
select * from Athlete where pays='France'



"""

In [16]:
# Afficher le nombre de n-uplets (par defaut)
execute(connection,query, show=False)

92 rows


<cursor object at 0x7f43f26ec8b0; closed: -1>

In [17]:
# Afficher le résultat (par defaut)
execute(connection,query)

92 rows
nom             |prenom          |datenaissance   |pays            |
--------------------------------------------------------------------
FOURCADE        |Martin          |1988-09-14      |France          |
LAMY CHAPPUIS   |Jason           |1986-09-09      |France          |
VAULTIER        |Pierre          |1987-06-24      |France          |
CHAPUIS         |Jean Frederic   |1989-03-02      |France          |
BRUNET          |Marie Laure     |1988-11-20      |France          |
DORIN HABERT    |Marie           |1986-06-19      |France          |
ANTHONIOZ       |Deborah         |1978-08-29      |France          |
MISSILLIER      |Steve           |1984-12-12      |France          |
BOVOLENTA       |Arnaud          |1988-09-06      |France          |
MARTINOD        |Marie           |1984-07-20      |France          |
GAILLARD        |Jean Marc       |1980-10-07      |France          |
DE LE RUE       |Paul-Henri      |1984-04-17      |France          |
PINTURAULT      |Alexis   

<cursor object at 0x7f43f26ec5e0; closed: -1>

**Q2**. La date de naissance de Denis TEN.<br/>
Résultat: 1993-06-13

In [18]:
query="""
SELECT dateNaissance
FROM Athlete
WHERE lower(prenom) = 'denis' AND lower(nom) = 'ten'

"""
execute(connection,query)

1 rows
datenaissance   |
-----------------
1993-06-13      |


<cursor object at 0x7f43f26ec9a0; closed: -1>

**Q3**. Les épreuves (sport, épreuve) dans lesquelles Therese JOHAUG a gagné une médaille, ainsi
que le rang qu'elle a obtenu.<br/>
Indice : pour gagner une médaille, il faut avoir été classé dans le rang 1, 2 ou 3.<br/>
Résultat : ('Ski de fond', '10km', 3) et ('Ski de fond', '30km', 2)

In [20]:
query="""
SELECT sport, epreuve, rang
FROM RangEpreuve
WHERE prenomAth = 'Therese' AND nomAth = 'JOHAUG' AND rang <= 3

"""
execute(connection,query)

2 rows
sport           |epreuve         |rang            |
---------------------------------------------------
Ski de fond     |10km            |3               |
Ski de fond     |30km            |2               |


<cursor object at 0x7f43f26ecc70; closed: -1>

**Q4**. La nationalité des athlètes Kamil STOCH et Suk-Hee SHIM.<br/>
Résultat : ('SHIM', 'Suk-Hee', 'Corée du Sud') et ('STOCH', 'Kamil', 'Pologne')

In [21]:
query="""
SELECT nom, prenom, pays
FROM Athlete
WHERE (prenom = 'Kamil' AND nom = 'STOCH') OR (prenom = 'Suk-Hee' AND nom = 'SHIM')
"""
execute(connection,query)

2 rows
nom             |prenom          |pays            |
---------------------------------------------------
STOCH           |Kamil           |Pologne         |
SHIM            |Suk-Hee         |Corée du Sud    |


<cursor object at 0x7f43f26ecb80; closed: -1>

**Q5**. Tous les sports des Jeux Olympiques.<br/>
Résultat : (12 lignes)

In [22]:
query="""
SELECT DISTINCT sport FROM RangEpreuve

"""
execute(connection,query)

12 rows
sport           |
-----------------
Biathlon        |
Combiné nordique|
Luge            |
Patinage artisti|
Patinage de vite|
Patinage de vite|
Saut à ski      |
Skeleton        |
Ski acrobatique |
Ski alpin       |
Ski de fond     |
Surf des neiges |


<cursor object at 0x7f43f26eca90; closed: -1>

**Q6**. Le nom des athlètes originaire de Scandinavie (pays : Danemark, Finlande, Norvège, Suède,
Islande) triés par le nom, puis par leur prénom.<br/>
Résultat : (191 lignes)

In [26]:
query="""
SELECT nom, prenom
FROM Athlete
WHERE pays IN (' Danemark', 'Finlande', 'Norvège', 'Suède', 'Islande')
ORDER BY nom, prenom

"""
execute(connection,query, show=False)

189 rows


<cursor object at 0x7f43f26ed300; closed: -1>

**Q7**. Le nom de sports contenant la chaîne de caractères 'ski' (par exemple, 'Ski de fond', 'Saut à
ski', …).<br/>
Résultat : Ski de fond, Ski alpin, Ski acrobatique, Saut à ski

In [24]:
query="""
SELECT DISTINCT sport FROM RangEpreuve WHERE lower(sport) LIKE '%ski%'

"""
execute(connection,query)

4 rows
sport           |
-----------------
Saut à ski      |
Ski acrobatique |
Ski alpin       |
Ski de fond     |


<cursor object at 0x7f43f26ece50; closed: -1>

**Q8**. Les athlètes (nom, prénom) dont la date de naissance n'est pas renseignée.<br/>
Résultat : (1516 lignes)

In [27]:
query="""
SELECT nom, prenom FROM Athlete WHERE dateNaissance IS NULL

"""
execute(connection,query, show=False)

1516 rows


<cursor object at 0x7f44083bff10; closed: -1>

**Q9**. Les athlètes qui ont participé à l'épreuve '10km' de 'Ski de fond' sans être disqualifié (rang
inconnu), triés par rang.<br/>
Résultat : (75 lignes) 

In [29]:
query="""
SELECT nomAth, prenomAth
FROM RangEpreuve
WHERE sport = 'Ski de fond' AND epreuve = '10km' AND rang IS NOT NULL
ORDER BY rang
"""
execute(connection,query)

75 rows
nomath          |prenomath       |
----------------------------------
KOWALCZYK       |Justyna         |
KALLA           |Charlotte       |
JOHAUG          |Therese         |
SAARINEN        |Aino Kaisa      |
BJOERGEN        |Marit           |
BOEHLER         |Stefanie        |
ZHUKOVA         |Natalia         |
NISKANEN        |Kerttu          |
WENG            |Heidi           |
LAHTEENMAKI     |Krista          |
TCHEKALEVA      |Yulia           |
WIKEN           |Emma            |
KUZIUKOVA       |Olga            |
KYLLOENEN       |Anne            |
ISHIDA          |Masako          |
LINDBORG        |Sara            |
IVANOVA         |Julia           |
BJORNSEN        |Sadie           |
JACOBSEN        |Astrid Uhrenhold|
HAAG            |Anna            |
VRABCOVA-NYVLTOV|Eva             |
SMUTNA          |Katerina        |
FESSEL          |Nicole          |
SHEVCHENKO      |Valentina       |
ZELLER          |Katrin          |
KUBINSKA        |Kornelia        |
AYMONIER    

<cursor object at 0x7f43f26ed120; closed: -1>

**Q10**. Les épreuves (sport, épreuve et categorie) qui ont eu lieu après le 21 février 2014.<br/>
Résultat : (4 lignes)

In [30]:
query="""
SELECT DISTINCT sport, epreuve, categorie
FROM RangEpreuve
WHERE dateDebut > '2014-02-21'

"""
execute(connection,query)

4 rows
sport           |epreuve         |categorie       |
---------------------------------------------------
Ski alpin       |Slalom          |Femmes          |
Ski de fond     |50km            |Hommes          |
Surf des neiges |Slalom parallèle|Femmes          |
Surf des neiges |Slalom parallèle|Hommes          |


<cursor object at 0x7f43f26ed4e0; closed: -1>

**Q11**. Les épreuves (sport, épreuve et categorie) qui ont duré entre 2 et 4 jours, ainsi que leur
durée exacte.<br/>
Résultat : (16 lignes)

In [31]:
query="""
SELECT DISTINCT sport, epreuve, categorie
FROM RangEpreuve
WHERE DATEDIFF(day, dateDebut, dateFin) +1 BETWEEN 2 AND 4

"""
execute(connection,query)

16 rows
sport           |epreuve         |categorie       |
---------------------------------------------------
Combiné nordique|Individuel LH   |Hommes          |
Luge            |Simple          |Femmes          |
Luge            |Simple          |Hommes          |
Patinage artisti|Individuel      |Femmes          |
Patinage artisti|Individuel      |Hommes          |
Patinage de vite|1000m           |Femmes          |
Patinage de vite|1000m           |Hommes          |
Patinage de vite|500m            |Femmes          |
Patinage de vite|500m            |Hommes          |
Saut à ski      |Grand tremplin i|Hommes          |
Saut à ski      |Tremplin normal |Hommes          |
Skeleton        |Individuel      |Femmes          |
Skeleton        |Individuel      |Hommes          |
Ski acrobatique |Bosses          |Femmes          |
Surf des neiges |Slopestyle      |Femmes          |
Surf des neiges |Slopestyle      |Hommes          |


<cursor object at 0x7f43f26ed6c0; closed: -1>

**Q12**. Les épreuves (sport, épreuve et catégorie) dont le final a eu lieu pendant un weekend.<br/>
Résultat : (26 lignes)

In [37]:
query="""
SELECT DISTINCT sport, epreuve, categorie
FROM RangEpreuve
WHERE DAYNAME(dateFin) = 'Saturday' or DAYNAME(dateFin) = 'Sunday'

"""
execute(connection,query)

26 rows
sport           |epreuve         |categorie       |
---------------------------------------------------
Biathlon        |10km            |Hommes          |
Biathlon        |7,5km           |Femmes          |
Luge            |Simple          |Hommes          |
Patinage de vite|1500m           |Femmes          |
Patinage de vite|1500m           |Hommes          |
Patinage de vite|3000m           |Femmes          |
Patinage de vite|5000m           |Hommes          |
Patinage de vite|1000m           |Hommes          |
Patinage de vite|1500m           |Femmes          |
Saut à ski      |Grand tremplin i|Hommes          |
Saut à ski      |Tremplin normal |Hommes          |
Skeleton        |Individuel      |Hommes          |
Ski acrobatique |Bosses          |Femmes          |
Ski alpin       |Descente        |Femmes          |
Ski alpin       |Slalom          |Femmes          |
Ski alpin       |Super G         |Femmes          |
Ski alpin       |Super G         |Hommes          |
Ski 

<cursor object at 0x7f43f26ed8a0; closed: -1>

**Q13**. Les athlètes (nom, prénom) femmes de nationalité française.<br/>
Résultat : (26 lignes)

In [48]:
query="""
SELECT distinct nomAth, prenomAth
FROM RangEpreuve, Athlete
WHERE nom = nomath and prenom = prenomath and categorie = 'Femmes' and pays = 'France' 

"""
execute(connection,query)

26 rows
nomath          |prenomath       |
----------------------------------
AYMONIER        |Celia           |
BARTHET         |Anne-Sophie     |
BAUD            |Adeline         |
BERTRAND        |Marion          |
BESCOND         |Anais           |
BONNEFOY        |Morgane         |
BRUNET          |Marie Laure     |
CARADEUX        |Anais           |
CHEVALIER       |Anais           |
CLAIR           |Julia           |
DAVID           |Ophelie         |
DORIN HABERT    |Marie           |
FAIVRE PICON    |Anouk           |
GRIMAL          |Clemence        |
HUGUE           |Coraline        |
JEAN            |Aurore          |
LEMARE          |Lea             |
MARCHAND-ARVIER |Marie           |
MARMOTTAN       |Anemone         |
MARTINOD        |Marie           |
MATTEL          |Coline          |
MEITE           |Mae Berenice    |
NOENS           |Nastasia        |
RODRIGUEZ       |Sophie          |
THOVEX          |Mirabelle       |
TRESPEUCH       |Chloe           |


<cursor object at 0x7f43f26eea70; closed: -1>

**Q14**. Les athlètes (nom, prénom) qui ont aujourd'hui moins de 30 ans et qui ont gagné une
médaille.<br/>
Résultat : depend de la date d'exécution 

In [52]:
query="""
SELECT distinct nom, prenom, datenaissance, rang
FROM Athlete, RangEpreuve
WHERE nom = nomath and prenom = prenomath and rang <= 3 and datediff(year,datenaissance,CURRENT_DATE)<30

"""
execute(connection,query)

8 rows
nom             |prenom          |datenaissance   |rang            |
--------------------------------------------------------------------
HIRANO          |Ayumu           |1998-11-29      |2               |
HIRAOKA         |Taku            |1995-10-29      |3               |
MATTEL          |Coline          |1995-11-03      |3               |
SHIFFRIN        |Mikaela         |1995-03-13      |1               |
SHIM            |Suk-Hee         |1997-01-30      |2               |
SHIM            |Suk-Hee         |1997-01-30      |3               |
SOTNIKOVA       |Adelina         |1996-07-01      |1               |
TIANYU          |Han             |1996-06-03      |2               |


<cursor object at 0x7f44083bf010; closed: -1>

**Q15**. Les athlètes (nom, prénom) qui avaient leur anniversaire à la date de fin de l'épreuve dans
laquelle ils participaient.<br/>
Résultat : ('GARANICHEV', 'Evgeniy', 13, 2), ('SKOBREV', 'Ivan', 8, 2) et ('WEIBRECHT',
'Andrew', 10, 2)

In [54]:
query="""
SELECT distinct nom, prenom
FROM Athlete, RangEpreuve
WHERE nom = nomath and prenom = prenomath and month(datenaissance) = month(datefin) and day_of_month(datenaissance) = day_of_month(datefin)
"""
execute(connection,query)

3 rows
nom             |prenom          |
----------------------------------
GARANICHEV      |Evgeniy         |
SKOBREV         |Ivan            |
WEIBRECHT       |Andrew          |


<cursor object at 0x7f43f26ef3d0; closed: -1>

**Q16**.  Les athlètes français (nom, prénom) qui ont gagné une médaille.<br/>
Résultat : (12 lignes)

In [55]:
query="""
SELECT distinct nomAth, prenomAth
FROM RangEpreuve, Athlete
WHERE nom = nomath and prenom = prenomath and rang<4 and pays = 'France' 

"""
execute(connection,query)

12 rows
nomath          |prenomath       |
----------------------------------
BEATRIX         |Jean Guillaume  |
BOVOLENTA       |Arnaud          |
CHAPUIS         |Jean Frederic   |
FOURCADE        |Martin          |
MARTINOD        |Marie           |
MATTEL          |Coline          |
MIDOL           |Jonathan        |
MISSILLIER      |Steve           |
PINTURAULT      |Alexis          |
ROLLAND         |Kevin           |
TRESPEUCH       |Chloe           |
VAULTIER        |Pierre          |


<cursor object at 0x7f43f26ef790; closed: -1>

**Q17**. Les sports qui n'ont pas d'épreuves féminines.<br/>
Résultat : Combiné nordique

In [76]:
query="""
SELECT DISTINCT sport
FROM RangEpreuve r1
WHERE not exists (select * from rangepreuve r2 where r1.sport = r2.sport and Categorie = 'Femmes')

"""
execute(connection,query)

1 rows
sport           |
-----------------
Combiné nordique|


<cursor object at 0x7f43f2504040; closed: -1>

## TME4 

## Charger les données bd_jo_v1_H2.sql

In [77]:
schemafile=open("bd-jo-v1_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


### Exercice 1

Exprimer les requêtes suivantes en SQL 

**Q1**. Les athlètes (nom, prénom) d'Inde (104 lignes).<br/>
Résultat : IQBAL Nadeem, KESHAVAN Shiva, THAKUR Himanshu

In [80]:
query="""
select nom, prenom 
from Athlete 
where pays='Inde'
"""
execute(connection,query)

3 rows
nom             |prenom          |
----------------------------------
IQBAL           |Nadeem          |
KESHAVAN        |Shiva           |
THAKUR          |Himanshu        |


<cursor object at 0x7f43f2504130; closed: -1>

**Q2** Le prénom des athlètes dont le nom est 'GOLD'.<br/>
Résultat : Arielle, Gracie, Taylor

In [81]:
query="""
select prenom 
from Athlete 
where nom='GOLD'

"""
execute(connection,query)

3 rows
prenom          |
-----------------
Arielle         |
Gracie          |
Taylor          |


<cursor object at 0x7f43f25046d0; closed: -1>

**Q3** La nationalité de AONO Ryo.<br/>
Résultat : Japon

In [82]:
query="""
select pays
from Athlete 
where nom = 'AONO' and prenom = 'Ryo'

"""
execute(connection,query)

1 rows
pays            |
-----------------
Japon           |


<cursor object at 0x7f43f25047c0; closed: -1>

**Q4** Le gagnant du médaille d'or de chaque épreuve. Triez le résultat par sport, puis par épreuve.<br/>
Résultat : (74 lignes)

In [87]:
query="""
select sport, epreuve, nomath, prenomath
from rangepreuve
where rang = 1
order by sport, epreuve
"""
execute(connection,query)

74 rows
sport           |epreuve         |nomath          |prenomath       |
--------------------------------------------------------------------
Biathlon        |10km            |BJOERNDALEN     |Ole Einar       |
Biathlon        |10km poursuite  |DOMRACHEVA      |Darya           |
Biathlon        |12,5km départ gr|DOMRACHEVA      |Darya           |
Biathlon        |12,5km poursuite|FOURCADE        |Martin          |
Biathlon        |15km            |DOMRACHEVA      |Darya           |
Biathlon        |15km départ grou|SVENDSEN        |Emil Hegle      |
Biathlon        |20km            |FOURCADE        |Martin          |
Biathlon        |7,5km           |DOMRACHEVA      |Darya           |
Combiné nordique|Individuel      |FRENZEL         |Eric            |
Combiné nordique|Individuel LH   |GRAABAK         |Joergen         |
Luge            |Simple          |GEISENBERGER    |Natalie         |
Luge            |Simple          |LOCH            |Felix           |
Patinage artisti|Individue

<cursor object at 0x7f43f25049a0; closed: -1>

**Q5** Toutes les épreuves (sport, épreuve, catégorie) triées par la date de fin de l'épreuve.   
Résultat : (73 lignes)

In [116]:
query="""
select sport, epreuve, nomath, prenomath
from rangepreuve
where rang = 1
order by sport, epreuve

"""
execute(connection,query)

The error 'connection already closed' occurred


**Q6** Les athlètes nés entre 1990 et 1995.<br/>
Résultat : (87 lignes)

In [90]:
query="""
select nom, prenom
from athlete
where datenaissance< '1995-12-31' and datenaissance> '1990-01-01'

"""
execute(connection,query)

87 rows
nom             |prenom          |
----------------------------------
ZHOU            |Yang            |
PARK            |Seung-Hi        |
GRAABAK         |Joergen         |
SCHLIERENZAUER  |Gregor          |
OESTBERG        |Ingvild Flugstad|
KIM             |Yuna            |
VERWEIJ         |Koen            |
ECKHOFF         |Tiril           |
BEEK            |Lotte van       |
DZHYMA          |Juliya          |
FALLA           |Maiken Caspersen|
ANDERSON        |Jamie           |
ELISTRATOV      |Semion          |
WELLINGER       |Andreas         |
NISKANEN        |Iivo            |
HANYU           |Yuzuru          |
KOTSENBURG      |Sage            |
SAMKOVA         |Eva             |
KIM             |Alang           |
VOGT            |Carina          |
KRAUS           |Marinus         |
MAYER           |Matthias        |
SHIFFRIN        |Mikaela         |
THOMPSON        |Marielle        |
DUFOUR-LAPOINTE |Justine         |
HOWELL          |Dara            |
WISE        

<cursor object at 0x7f43f2504b80; closed: -1>

**Q7** Les athlètes suisses ayant participé au sport 'Biathlon' et disqualifié à au moins une épreuve
dece sport (solution sans jointure, avec une opération ensembliste).<br/>
Résultat : GASPARIN Elisa

In [94]:
query="""
select distinct nomath, prenomath
from rangepreuve
where sport = 'Biathlon' and rang is NULL and exists(select * from athlete where nom=nomath and prenom=prenomath and pays='Suisse')
"""
execute(connection,query)

1 rows
nomath          |prenomath       |
----------------------------------
GASPARIN        |Elisa           |


<cursor object at 0x7f43f2504d60; closed: -1>

**Q8** Les épreuves dans lesquelles il n'y a pas eu une médaille d'argent (solution sans jointure,
avec une opération ensembliste). <br/>
Résultat : Ski alpin Descente Femmes

In [96]:
query="""
select distinct sport, epreuve, categorie
from rangepreuve r1
where not exists(select * from rangepreuve r2 where r1.sport = r2.sport and r1.epreuve = r2.epreuve and r1.categorie = r2.categorie and rang = 2)

"""
execute(connection,query)

1 rows
sport           |epreuve         |categorie       |
---------------------------------------------------
Ski alpin       |Descente        |Femmes          |


<cursor object at 0x7f43f2505120; closed: -1>

### Exercice 2

On considère maintenant le schéma complet de la base JO2014

**PAYS** (<u>CODEPAYS</u>, NOMP)<br/>
**SPORT** (<u>SID</u>, NOMSP)<br/>
**EPREUVE** (<u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN) <br/>
**ATHLETE** (<u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*) EQUIPE (EQID, CODEPAYS*)<br/>
**ATHLETESEQUIPE** (<u>EQID*, AID*</u>)<br/>
**RANGINDIVIDUEL** (<u>EPID*, AID*</u>, RANG)<br/>
**RANGEQUIPE** (<u>EPID*, EQID*</u>, RANG)<br/>
    

La relation PAYS contient le code et le nom de tous les pays, même si ils n'ont pas participé aux Jeux Olympiques. Les sports (n-uplets de la relation SPORT) sont un ensemble d'épreuves (n-uplets de la relation EPREUVE). Pour chaque épreuve on connaît son nom et les date de début et fin de l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table RANGINDIVIDUEL qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les épreuves par équipe les résultats sont stockés dans la relation RANGEQUIPE, alors que l'information sur le pays de chaque équipe et ses participants et stocké dans les relations EQUIPE et ATHLETESEQUIPE. Dans les relations RANGINDIVIDUEL et RANGEQUIPE l'attribut rang est égal à null si l'athlète ou l'équipe a été disqualifié.


## Charger les données bd_jo_v1_H2.sql

In [97]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


#### Requêtes

Exprimer les requêtes suivantes en SQL :

**Q9**. Les épreuves (sport, épreuve, catégorie) individuelles.<br/>
Résultat : (73 lignes)

In [100]:
query="""
select distinct s.sid, e.nomep, e.categorie
from sport s, epreuve e, rangindividuel r
where s.sid = e.sid and e.epid = r.epid
"""
execute(connection,query)


73 rows
sid             |nomep           |categorie       |
---------------------------------------------------
1               |10km            |Hommes          |
1               |10km poursuite  |Femmes          |
1               |12,5km départ gr|Femmes          |
1               |12,5km poursuite|Hommes          |
1               |15km            |Femmes          |
1               |15km départ grou|Hommes          |
1               |20km            |Hommes          |
1               |7,5km           |Femmes          |
3               |Individuel      |Hommes          |
3               |Individuel LH   |Hommes          |
6               |Simple          |Femmes          |
6               |Simple          |Hommes          |
7               |Individuel      |Femmes          |
7               |Individuel      |Hommes          |
8               |10000m          |Hommes          |
8               |1000m           |Femmes          |
8               |1000m           |Hommes          |
8   

<cursor object at 0x7f43f2505210; closed: -1>

**Q10**. Les résultats (nom, prénom, pays de l'athlète et rang) de l'épreuve '1000m' du 'Patinage de vitesse' pour les 'Femmes'.<br/>
Résultat : (36 lignes)

In [110]:
query="""
select a.nomath, a.prenomath, p.nomp, r.rang
from epreuve e, rangindividuel r, athlete a, sport s, pays p
where s.nomsp = 'Patinage de vitesse' and e.sid = s.sid and e.nomep = '1000m' and e.categorie = 'Femmes' and e.epid = r.epid and a.aid = r.aid and p.codepays = a.codepays
order by rang
"""
execute(connection,query)

36 rows
nomath          |prenomath       |nomp            |rang            |
--------------------------------------------------------------------
HONG            |Zhang           |République popul|1               |
WÜST            |Ireen           |Pays-Bas        |2               |
BOER            |Margot          |Pays-Bas        |3               |
FATKULINA       |Olga            |Russie          |4               |
BEEK            |Lotte van       |Pays-Bas        |5               |
LEENSTRA        |Marrit          |Pays-Bas        |6               |
RICHARDSON      |Heather         |États-Unis      |7               |
BOWE            |Brittany        |États-Unis      |8               |
NESBITT         |Christine       |Canada          |9               |
ERBANOVA        |Karolina        |République tchèq|10              |
HESSE           |Judith          |Allemagne       |11              |
SANG-HWA        |Lee             |Corée du Sud    |12              |
KODAIRA         |Nao      

<cursor object at 0x7f43f2506200; closed: -1>

**Q11**. Le nom et prénom des athlètes qui forment l'équipe qui a gagné la médaille d'or dans l'épreuve 'relais 4x6km' de 'Biathlon' de 'Femmes'.   
Résultat : SEMERENKO Vita, SEMERENKO Valj, DZHYMA Juliya, PIDHRUSHNA Olena

In [114]:
query="""
select a.nomath, a.prenomath
from epreuve e, rangequipe r, athlete a, athletesEquipe aEq, sport s
where s.nomsp = 'Biathlon' and e.sid = s.sid and e.nomep = 'relais 4x6km' and e.categorie = 'Femmes' and e.epid = r.epid and r.rang = 1 and r.eqid = aEq.eqid and aEq.aid = a.aid

"""
execute(connection,query)

4 rows
nomath          |prenomath       |
----------------------------------
SEMERENKO       |Vita            |
SEMERENKO       |Valj            |
DZHYMA          |Juliya          |
PIDHRUSHNA      |Olena           |


<cursor object at 0x7f43f25062f0; closed: -1>

# Fermer la connexion

In [115]:
connection.close()